In [1]:
import json
import os
import pandas as pd
from utils import get_auth_code, get_access_token
import business_api_client
import json
import pandas as pd
import requests
from constants import URL_ADVERTISER_IDS, URL_ADVERTISER_INFO
from datetime import date, timedelta
from dotenv import load_dotenv

# 0 - Load env for TikTok Business API

In [2]:
load_dotenv()

redirect_uri = os.environ.get("REDIRECT_URI", "")
app_id = os.environ.get("APP_ID", "")
secret = os.environ.get("SECRET", "")
bc_id = os.environ.get("BC_ID", "")

# Get auth code

In [3]:
authorize_url = "https://business-api.tiktok.com/portal/auth"

In [4]:
auth_code = get_auth_code(app_id, redirect_uri, authorize_url)

Generating authorization URL...
URL: https://business-api.tiktok.com/portal/auth?app_id=7332410169608896514&response_type=code&redirect_uri=https%3A%2F%2Fwww.paloceras.com&state=193e2aee2e5a7615afe8cdf3b7010156
State: 193e2aee2e5a7615afe8cdf3b7010156
Paste the URL you were redirected to


# Get access token

In [7]:
access_token_url = (
    "https://business-api.tiktok.com/open_api/v1.3/oauth2/access_token/"
)


In [8]:
my_args = '{"secret": "%s", "app_id": "%s", "auth_code": "%s"}' % (
    secret,
    app_id,
    auth_code,
)

access_token = get_access_token(my_args, access_token_url)["data"]["access_token"]
access_token

'b2ab0cf5687a998933f1e357a5ed3cc884ef64b5'

In [9]:
response_advertiser_ids = requests.get(
    url=URL_ADVERTISER_IDS,
    headers={"Access-Token": access_token},
    params={"app_id": app_id, "secret": secret},
)

api_report = business_api_client.ReportingApi()

advertiser_ids = response_advertiser_ids.json()["data"]["list"]

ad_ids = []
for advertiser in advertiser_ids:
    advertiser_id = advertiser["advertiser_id"]
    ad_ids.append(advertiser_id)

response_advertiser_info = requests.get(
    url=URL_ADVERTISER_INFO,
    headers={"Access-Token": access_token},
    params={"advertiser_ids": json.dumps(ad_ids)},
)
response_advertiser_info = response_advertiser_info.json()["data"]["list"]

date_from = date.today().replace(day=1)
date_to = (date_from + timedelta(days=32)).replace(day=1) - timedelta(days=1)
metrics = [
    "spend",
    "clicks",
    "impressions",
    "cost_per_result",
    "reach",
    "cpc",  # cost_per_result: Cost
    "cpm",
    "ctr",
    "conversion",
    "cost_per_conversion",  # cost_per_conversion: CPA
]

for advertiser_id in ad_ids:
    api_report_responses = api_report.report_integrated_get(
        advertiser_id=advertiser_id,
        report_type="BASIC",
        access_token=access_token,
        dimensions=["ad_id"],
        data_level="AUCTION_AD",
        metrics=metrics,
        start_date=date_from,
        end_date=date_to,
    )

    for advertiser in response_advertiser_info:
        if (
            "owner_bc_id" in advertiser
            and advertiser["owner_bc_id"] == bc_id
            and advertiser["advertiser_id"] == advertiser_id
        ):

            if api_report_responses["data"]["list"] == []:

                api_report_responses["data"]["list"] = [
                    {
                        "Account_Name": advertiser["name"],
                        "Account_ID": advertiser["advertiser_id"],
                        "Clicks": 0,
                        "Impression": 0,
                        "Cost": 0,
                        "Reach": 0,
                        "CPC": 0,
                        "CPM": 0,
                        "CTR": 0,
                        "Conversion": 0,
                        "CPA": 0,
                    }
                ]

            api_report_response_results = api_report_responses["data"]["list"]

            df = pd.DataFrame(api_report_response_results)
            df.rename(
                columns={"cost_per_result": "cost", "cost_per_conversion": "cpa"},
                inplace=True,
            )
            # df.to_csv("tiktok_business_report.csv", index=False)
df

,Account_Name,Account_ID,Clicks,Impression,Cost,Reach,CPC,CPM,CTR,Conversion,CPA
0,Test Account Vjppr0,7332700118070116354,0,0,0,0,0,0,0,0,0
